# HW8 (20')

<font size='4'>

For this assignment, it is a combination of jupyter notebook assignment and python scripts.

For Q1, please upload your outputs including codes and graphics to your own GitHub repository. <br> You will need to disclose your GitHub repository below.

For Q2, please submit this jupyter notebook as an HTML or PDF file.

First of all, print your name (First and Last) below.

In [1]:
print("TANMAYEE KODALI")

TANMAYEE KODALI


## 0. Import relevant packages

In [3]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.formula.api as smf

## Q1. Convert your HW7 to python scripts. (10')

<font size='4'>

- Under your working directory, there should be a folder called `self_py_fun`.
- Create a new python file called `HW8Fun.py` and move previously defined functions `produce_trun_mean_cov()`, `plot_trunc_mean()`, and `plot_trunc_cov()` to that file. Make sure you import proper packages.
- Create another main file `HW8_main.py`.
- Import relevant packages, modules, and/or function.
- Copy the global variables and call your functions inside `HW8_main.py`.
- A major difference compared to HW7 is that you are asked to save those figures to your local working environment.
    - Create a new directory `K114` under your current working directory.
    - For mean functions, please save it as a `Mean.png` output using `plt.savefig()` function.
    - The changes should be made within `HW8Fun.py` rather than `HW8_main.py`.
    - For covariance matrices, please save them as `Covariance_Target.png`, `Covariance_Non-Target.png`, and `Covariance_All.png` outputs using the same function above.
    - To summarize, there should be **four** figures under `K114` folder.
- Upload your entire work to your GitHub repository via push button.

In [90]:
# Provide your GitHub repository link below in the Markdown chunk. Remember to make it public and make the link clickable.
# Do not include sensitive information in your GitHub repository.

https://github.com/TanmayeeKodali/BIOS-584

## Q2. A real-world data anlaysis using `Pandas` and `Scipy` (10')

<font size='4'>

- Back to the `PTSD dataset.xlsx`, let's import the dataset and name it `ptsd_df`. (no point since everyone has done it a couple of times before.)

In [8]:
# Write your own code
print(os.getcwd())
ptsd_dir = '{}/data/PTSD dataset.xlsx'.format(os.getcwd())
ptsd_df = pd.read_excel(ptsd_dir, sheet_name='main_dataset')
print(type(ptsd_df))
print(ptsd_df.head())

E:\Git Hub\BIOS-584
<class 'pandas.core.frame.DataFrame'>
   record_id  ptsdpresent_caps  caps_minuspcl  caps_minuspcl_code  \
0        460                 1            -56                   1   
1       5942                 1            -49                   1   
2       5366                 1            -48                   1   
3       3480                 1            -47                   1   
4       1029                 1            -46                   1   

   caps_intake  pcl5_score_intake  pcl5month_score.baseline  \
0           15                 71                      75.0   
1           23                 72                       NaN   
2           31                 79                      62.0   
3           31                 78                      76.0   
4           28                 74                      52.0   

   pcl5week_score.completion  pcl5month_score.3_month_follow_up  mdd_code  \
0                       40.0                                NaN        

### Q2.1. Univariate comparison (3')

<font size='4'> 
    
- Suppose that we would like to examine the utility/effect of an intervention program for patients with PTSD.
- We measure PCL5 scores at completion (`pcl5week_score.completion`) and PCL5 score at 3-month follow-up (`pcl5month_score.3_month_follow_up`). Let's assume the first score is pre-intervention and the second score is post-intervention.
- Report the summary statistics for each variable including mean, std, median, Q1, and Q3.
- Note that each patient will receive such two PCL5 scores. Use a appropriate statistical test to perform the univariate comparison. Report the outputing statistic and p-value.
- Before you run the statistic test, determine the data type and check the missingness of two columns. In particular, report the number of NA values for each variable.

In [11]:
# Write your own code
# Define the two variables
pre_int = 'pcl5week_score.completion'
post_int = 'pcl5month_score.3_month_follow_up'

# Check data types
print(ptsd_df[pre_int].dtype)
print(ptsd_df[post_int].dtype)

# Check missing values
print(ptsd_df[pre_int].isna().sum())
print(ptsd_df[post_int].isna().sum())

# Remove rows with missing values
df_clean = ptsd_df[[pre_int, post_int]].dropna()

# Summary statistics
pre_stats = df_clean[pre_int].describe()
post_stats = df_clean[post_int].describe()

print(pre_stats)
print(post_stats)

# Paired t-test (using your stats import)
t_statistic, p_value = stats.ttest_rel(df_clean[pre_int], df_clean[post_int])

print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

float64
float64
27
251
count    224.000000
mean      27.209821
std       19.184862
min        0.000000
25%       12.750000
50%       22.000000
75%       41.000000
max       75.000000
Name: pcl5week_score.completion, dtype: float64
count    224.000000
mean      32.026786
std       18.978610
min        0.000000
25%       16.000000
50%       31.000000
75%       46.000000
max       79.000000
Name: pcl5month_score.3_month_follow_up, dtype: float64
t-statistic: -4.709238006778945
p-value: 4.370038747681667e-06


### Q2.2. Multiple Linear Regression (7')

<font size='4'>

- Select columns specified in the following code chunk and create a subset dataset named `ptsd_sub_df`.
- Fit a linear regression to examine the association between `caps_intake` (continuous outcome) and the remaining covariates (as predictors) using `ptsd_sub_df`.
    - Note that all covariates ending with `_code` are categorical variables.
- Use the instruction here to write the formula for linear regression in Python.
    - https://www.statsmodels.org/stable/example_formulas.html
- Report the output page including R2, adjusted R2, and parameter estimates, SE, 95% confidence intervals, and p-values.
- Provide a brief interpretation for all significant predictors (p<0.05) excluding the intercept.
- Relevant label information includes:
    - `employment_code`: 1: Employed, 2: Unemployed, 3: Retired, 4: Disabled/Unable to work, 5: Student, 6: Other.
    - `rank_code`: 1. Enlisted, 2: Officer, 3: Other

In [22]:
# The following column names are used for linear regression.
# Do not delete.
relevant_col_names = ['caps_intake', 'age_iop', 'gender_code', 'sexualorient_code', 'race_code', 'ethnicity_code', 
                      'education_code', 'employment_code',
                      'rank_code', 'branch_code', 'mdd_code', 'ctq_total_score', 'sexual_trauma', 'sud_code']
# Create subset dataset
ptsd_sub_df = ptsd_df[relevant_col_names].copy()

# Check for missing values
print("Missing values in subset:", 
      ptsd_sub_df.isna().sum())

# Remove rows with missing values
ptsd_sub_df_clean = ptsd_sub_df.dropna()
print(f"\nRows after removing missing values: {len(ptsd_sub_df_clean)}")

Missing values in subset: caps_intake           0
age_iop               0
gender_code           0
sexualorient_code     0
race_code             0
ethnicity_code        0
education_code        0
employment_code       0
rank_code             0
branch_code           0
mdd_code              0
ctq_total_score      35
sexual_trauma         0
sud_code              0
dtype: int64

Rows after removing missing values: 448


In [19]:
# Write your own code
#Convert categorical variables to category type
categorical_vars = ['gender_code', 'sexualorient_code', 'race_code', 'ethnicity_code', 
                    'education_code', 'employment_code', 'rank_code', 'branch_code', 
                    'mdd_code', 'sud_code']

for var in categorical_vars:
    ptsd_sub_df_clean.loc[:, var] = ptsd_sub_df_clean[var].astype('category')

print(f"\nCategorical variables: {categorical_vars}")

# Fit linear regression using statsmodels formula
# C() indicates categorical variables
formula = '''caps_intake ~ age_iop + C(gender_code) + C(sexualorient_code) + C(race_code) + 
             C(ethnicity_code) + C(education_code) + C(employment_code) + C(rank_code) + 
             C(branch_code) + C(mdd_code) + ctq_total_score + sexual_trauma + C(sud_code)'''

# Fit the model
model = smf.ols(formula=formula, data=ptsd_sub_df_clean).fit()

# Display full output
print("\n" + "="*80)
print("LINEAR REGRESSION RESULTS")
print("="*80)
print(model.summary())



Categorical variables: ['gender_code', 'sexualorient_code', 'race_code', 'ethnicity_code', 'education_code', 'employment_code', 'rank_code', 'branch_code', 'mdd_code', 'sud_code']

LINEAR REGRESSION RESULTS
                            OLS Regression Results                            
Dep. Variable:            caps_intake   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     1.796
Date:                Tue, 28 Oct 2025   Prob (F-statistic):            0.00697
Time:                        23:30:55   Log-Likelihood:                -1569.0
No. Observations:                 448   AIC:                             3200.
Df Residuals:                     417   BIC:                             3327.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
  

# Write your interpretations below:
Only three predictors are found to have significant (p<0.05) values: 
1. C(employment_code)[T.4] (p = 0.029)
2. C(rank_code)[T.2] (p = 0.022)
3. ctq_total_score (p = 0.035)


Interpretations:
1. Compared to Employed individuals, those who are Disabled/Unable to work have 2.74 points higher CAPS intake scores on average, holding other variables constant. This suggests greater PTSD symptom severity among those unable to work due to disability.
2. Compared to Enlisted personnel, Officers have 3.06 points lower CAPS intake scores on average, holding other variables constant. Officers report less severe PTSD symptoms than Enlisted personnel.
3. For each 1-point increase in CTQ total score (childhood trauma), CAPS intake score increases by 0.042 points, holding other variables constant. Higher childhood trauma is associated with more severe PTSD symptoms at intake.